# Part 1 → RNN


## Sentiment Analysis with RNN on Amazon Fine Food Reviews

## Dataset
- **Amazon Fine Food Reviews**  
- [Download from Kaggle](https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews)  
- Dataset contains **500,000+ reviews** with ratings (1–5 stars).

## Objective
- Build a **Recurrent Neural Network (RNN)** model to predict review sentiment:
  - **Multi-class** → Ratings from 1 to 5 stars  


## Steps

### 1. Data Preprocessing
- Load the CSV file → focus on **`Text`** + **`Score`** columns.
- Clean text: lowercase, remove punctuation, (optional: remove stopwords).
- Tokenize reviews (convert words to numbers).
- Pad sequences to a fixed length.
- Convert labels:
  - Multi-class (1–5 → one-hot encoded).

### 2. Build RNN Model
- Use **Embedding Layer** to convert tokens into vectors.
- Add **SimpleRNN layer** (e.g., 32 or 64 units).
- Add **Dense output layer**:
  - Activation = `softmax` (for multi-class classification).

### 3. Training
- Compile with:
  - Optimizer → `adam`
  - Loss → `categorical_crossentropy` (multi-class)
  - Metric → `accuracy`
- Train for **5–10 epochs**.
- Use a **validation split** (e.g., 20%).

### 4. Evaluation
- Evaluate model on test set.
- Show:
  - **Accuracy score**
  - **Confusion matrix**
- Print a few **example predictions**:
  - Input review
  - True label
  - Predicted sentiment

### 5. User Input Prediction
- After training, let the user type **any review sentence**.  
- Preprocess it the same way (tokenize + pad).  
- Pass it to the trained RNN model.  
- Print the predicted sentiment (Rating 1–5).  

## Hints
- Start with a **small subset** of the dataset (e.g., 20k reviews) to save training time.
- Watch for **overfitting** (training accuracy much higher than validation).

In [2]:
# Install kagglehub (only once per Colab session)
!pip install -q kagglehub[pandas-datasets]

import kagglehub
from kagglehub import KaggleDatasetAdapter

# Path to the dataset file inside the Kaggle dataset
file_path = "Reviews.csv"   # <- important, dataset contains this CSV file

# Load dataset directly as a pandas DataFrame
df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "snap/amazon-fine-food-reviews",  # dataset slug
    file_path
)

print("Shape:", df.shape)
print(df.head())


/tmp/ipython-input-87149450.py:11: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(


Using Colab cache for faster access to the 'amazon-fine-food-reviews' dataset.
Shape: (568454, 10)
   Id   ProductId          UserId                      ProfileName  \
0   1  B001E4KFG0  A3SGXH7AUHU8GW                       delmartian   
1   2  B00813GRG4  A1D87F6ZCVE5NK                           dll pa   
2   3  B000LQOCH0   ABXLMWJIXXAIN  Natalia Corres "Natalia Corres"   
3   4  B000UA0QIQ  A395BORC6FGVXV                             Karl   
4   5  B006K2ZZ7K  A1UQRSCLF8GW1T    Michael D. Bigham "M. Wassir"   

   HelpfulnessNumerator  HelpfulnessDenominator  Score        Time  \
0                     1                       1      5  1303862400   
1                     0                       0      1  1346976000   
2                     1                       1      4  1219017600   
3                     3                       3      2  1307923200   
4                     0                       0      5  1350777600   

                 Summary                                   

In [3]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
subset, _ = train_test_split(
    df[["Text", "Score"]],
    train_size=40000,
    stratify=df["Score"],
    random_state=42
)

# Step 3: Train/test split (80/20) again
train_texts, test_texts, train_labels, test_labels = train_test_split(
    subset["Text"],
    subset["Score"],
    test_size=0.2,
    stratify=subset["Score"],
    random_state=42
)

# Step 4: Shuffle
train_texts, train_labels = shuffle(train_texts, train_labels, random_state=42)
test_texts, test_labels = shuffle(test_texts, test_labels, random_state=42)


In [4]:
train_texts

,Text
305132,I usually find these at my local health food s...
114315,Eagle Pack has turned my picky eater into an e...
374696,C&P Organix is a great line of dog food. I'm ...
238117,This is the sugar I have come to use for every...
324513,"I originally gave 5 star rating, but none of t..."
...,...
402115,These are so good. My kids have eaten them si...
73069,The taste reminds me of when you accidentally ...
134409,Our Pom hates taking pills - hide it in someth...
458767,The directions on the bottle state that one sh...


In [5]:
import re
def clean_text(sen):
    sen=str(sen).lower()
    sen=re.sub('[^\w\s]','',sen)
    sen=re.sub('\d','',sen)
    return sen

train_texts=train_texts.apply(clean_text)
test_texts=test_texts.apply(clean_text)

<>:4: SyntaxWarning: invalid escape sequence '\w'
<>:5: SyntaxWarning: invalid escape sequence '\d'
<>:4: SyntaxWarning: invalid escape sequence '\w'
<>:5: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipython-input-3822074521.py:4: SyntaxWarning: invalid escape sequence '\w'
  sen=re.sub('[^\w\s]','',sen)
/tmp/ipython-input-3822074521.py:5: SyntaxWarning: invalid escape sequence '\d'
  sen=re.sub('\d','',sen)


In [6]:
def remove_punctuation(text):
    return re.sub(r"[^\w\s]", "", text)

train_texts=train_texts.apply(remove_punctuation)
test_texts=test_texts.apply(remove_punctuation)


In [7]:
train_texts

,Text
305132,i usually find these at my local health food s...
114315,eagle pack has turned my picky eater into an e...
374696,cp organix is a great line of dog food im not...
238117,this is the sugar i have come to use for every...
324513,i originally gave star rating but none of the...
...,...
402115,these are so good my kids have eaten them sin...
73069,the taste reminds me of when you accidentally ...
134409,our pom hates taking pills hide it in somethi...
458767,the directions on the bottle state that one sh...


In [8]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
def stop(text):
  words = text.split()
  words = [w for w in words if w not in stop_words]  # optional
  return " ".join(words)

train_texts = [stop(t) for t in train_texts]
test_texts = [stop(t) for t in test_texts]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_texts)
X_test = tokenizer.texts_to_sequences(test_texts)
word_index=tokenizer.word_index
X_train = tokenizer.texts_to_sequences(train_texts)
padded_sequences_train = pad_sequences(X_train)
padded_sequences_test = pad_sequences(X_test)




In [10]:
from tensorflow.keras.utils import to_categorical

train_labels = train_labels - 1
test_labels = test_labels - 1

train_labels = to_categorical(train_labels, num_classes=5)
test_labels = to_categorical(test_labels, num_classes=5)

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense,Dropout
rnn_model=Sequential([Embedding(input_dim=len(word_index)+1,output_dim=64,input_length=padded_sequences_train.shape[1]),
                      SimpleRNN(64),
                      Dropout(0.5),
                      Dense(5,activation="softmax")])
rnn_model.compile(optimizer="adam",loss="crossentropy",metrics=["accuracy"])
rnn_model.fit(
    padded_sequences_train, train_labels,
    validation_data=(padded_sequences_test, test_labels),
    batch_size=64,
    epochs=5
)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 212s 418ms/step - accuracy: 0.6281 - loss: 1.1177 - val_accuracy: 0.6840 - val_loss: 0.8795
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 189s 377ms/step - accuracy: 0.7305 - loss: 0.7371 - val_accuracy: 0.6944 - val_loss: 0.9059
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 205s 384ms/step - accuracy: 0.8662 - loss: 0.4118 - val_accuracy: 0.6511 - val_loss: 1.0236
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 190s 380ms/step - accuracy: 0.9495 - loss: 0.1775 - val_accuracy: 0.6420 - val_loss: 1.2631
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 191s 383ms/step - accuracy: 0.9787 - loss: 0.0781 - val_accuracy: 0.6447 - val_loss: 1.4783


In [12]:
import numpy as np
y_pred_probs = rnn_model.predict(padded_sequences_test)
y_pred = np.argmax(y_pred_probs, axis=1)   # predicted class (0–4)
y_true = np.argmax(test_labels, axis=1)

250/250 ━━━━━━━━━━━━━━━━━━━━ 9s 35ms/step


In [13]:
for i in range(60):
    print(f"Review {i+1}: True={y_true[i]}, Pred={y_pred[i]}")

Review 1: True=4, Pred=4
Review 2: True=4, Pred=4
Review 3: True=4, Pred=4
Review 4: True=4, Pred=4
Review 5: True=4, Pred=4
Review 6: True=4, Pred=4
Review 7: True=4, Pred=4
Review 8: True=4, Pred=4
Review 9: True=2, Pred=0
Review 10: True=4, Pred=4
Review 11: True=4, Pred=4
Review 12: True=4, Pred=3
Review 13: True=0, Pred=0
Review 14: True=1, Pred=2
Review 15: True=4, Pred=3
Review 16: True=3, Pred=4
Review 17: True=0, Pred=0
Review 18: True=4, Pred=4
Review 19: True=4, Pred=3
Review 20: True=1, Pred=1
Review 21: True=4, Pred=4
Review 22: True=4, Pred=4
Review 23: True=4, Pred=4
Review 24: True=3, Pred=4
Review 25: True=4, Pred=4
Review 26: True=4, Pred=4
Review 27: True=4, Pred=4
Review 28: True=4, Pred=4
Review 29: True=4, Pred=4
Review 30: True=4, Pred=4
Review 31: True=0, Pred=0
Review 32: True=4, Pred=4
Review 33: True=4, Pred=4
Review 34: True=4, Pred=4
Review 35: True=3, Pred=4
Review 36: True=4, Pred=4
Review 37: True=4, Pred=4
Review 38: True=4, Pred=4
Review 39: True=4, Pr

In [39]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

def print_score(model, X_train, y_train, X_test, y_test, train=True):
    if train:
        # Convert labels (one-hot → integers)
        y_true = np.argmax(y_train, axis=1)
        # Predictions (probabilities → integers)
        y_pred = np.argmax(model.predict(X_train), axis=1)

        clf_report = pd.DataFrame(classification_report(y_true, y_pred, output_dict=True))
        print("Train Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(y_true, y_pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n{confusion_matrix(y_true, y_pred)}\n")

    else:
        y_true = np.argmax(y_test, axis=1)
        y_pred = np.argmax(model.predict(X_test), axis=1)

        clf_report = pd.DataFrame(classification_report(y_true, y_pred, output_dict=True))
        print("Test Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(y_true, y_pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n{confusion_matrix(y_true, y_pred)}\n")


In [40]:
print_score(rnn_model, padded_sequences_train, train_labels, padded_sequences_test, test_labels, train=True)


1000/1000 ━━━━━━━━━━━━━━━━━━━━ 56s 56ms/step
Train Result:
Accuracy Score: 99.53%
_______________________________________________
CLASSIFICATION REPORT:
                     0            1            2            3             4  \
precision     0.992545     0.996379     0.992899     0.987947      0.997553   
recall        0.995581     0.985084     0.990417     0.992952      0.997212   
f1-score      0.994061     0.990699     0.991656     0.990443      0.997382   
support    2942.000000  1676.000000  2400.000000  4540.000000  20442.000000   

           accuracy     macro avg  weighted avg  
precision  0.995313      0.993465      0.995319  
recall     0.995313      0.992249      0.995313  
f1-score   0.995313      0.992848      0.995313  
support    0.995313  32000.000000  32000.000000  
_______________________________________________
Confusion Matrix: 
[[ 2929     1     3     2     7]
 [    5  1651     5     8     7]
 [    3     2  2377     5    13]
 [    4     2     3  4508    23]
 [

In [41]:
print_score(rnn_model, padded_sequences_train, train_labels, padded_sequences_test, test_labels, train=False)

250/250 ━━━━━━━━━━━━━━━━━━━━ 9s 36ms/step
Test Result:
Accuracy Score: 64.48%
_______________________________________________
CLASSIFICATION REPORT:
                    0           1           2            3            4  \
precision    0.481752    0.273529    0.292105     0.304233     0.791322   
recall       0.538043    0.221957    0.185000     0.303965     0.824462   
f1-score     0.508344    0.245059    0.226531     0.304099     0.807552   
support    736.000000  419.000000  600.000000  1135.000000  5110.000000   

           accuracy    macro avg  weighted avg  
precision   0.64475     0.428588      0.629175  
recall      0.64475     0.414685      0.644750  
f1-score    0.64475     0.418317      0.635560  
support     0.64475  8000.000000   8000.000000  
_______________________________________________
Confusion Matrix: 
[[ 396   73   54   55  158]
 [ 131   93   42   44  109]
 [  92   64  111  102  231]
 [  60   49   68  345  613]
 [ 143   61  105  588 4213]]



In [15]:
import pandas as pd
new_sentence=['this product is bad and horrible i hate it','Good Quality and very Delight and Great product']
new_sentence=pd.Series(new_sentence)
new_sentence=new_sentence.apply(clean_text)
new_sentence=new_sentence.apply(remove_punctuation)
new_sentence=[stop(t) for t in new_sentence]
new_sequences=tokenizer.texts_to_sequences(new_sentence)
new_padded_sequences=pad_sequences(new_sequences,maxlen=padded_sequences_train.shape[1])
predictions=rnn_model.predict(new_padded_sequences)
predicted_classes = np.argmax(predictions, axis=1)
print(predicted_classes)



1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 575ms/step
[0 4]


# Part 2 – LSTM
1. **Build LSTM Model**
   - Keep preprocessing exactly the same.
   - Replace `SimpleRNN` with `LSTM` (e.g., 64–128 units).
   - Dense output (sigmoid or softmax).

2. **Training**
   - Same setup (adam, crossentropy, accuracy).
   - Train for 5–10 epochs.

3. **Evaluation**
   - Compare accuracy with RNN model.
   - Show confusion matrix.
   - Example predictions.

4. **User Input Prediction. **
   - Take a sentence from user.
   - Preprocess (tokenize + pad).
   - Predict sentiment using the LSTM model.

## Hints
- Use a **smaller subset** (e.g., 20k reviews) for quick experiments.
- Compare:
  - Training speed (RNN faster, LSTM slower).
  - Accuracy on long reviews.

## Deliverables
1. Notebook with:
   - RNN implementation
   - LSTM implementation
2. Final accuracy + confusion matrix for both models.
3. Comparison table (RNN vs LSTM).
4. Demo that accepts **any user sentence** and predicts sentiment with both models.

In [16]:
from tensorflow.keras.layers import LSTM
LSTM_model = Sequential([
    Embedding(input_dim=len(word_index)+1,output_dim=64,input_length=padded_sequences_train.shape[1]),
    LSTM(128),
    Dropout(0.5),
    Dense(5, activation="softmax")
])

LSTM_model.compile(
    optimizer="adam",
    loss="crossentropy",
    metrics=["accuracy"]
)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [17]:
LSTM_model.fit(
    padded_sequences_train, train_labels,
    validation_data=(padded_sequences_test, test_labels),
    batch_size=64,
    epochs=5
)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 1014s 2s/step - accuracy: 0.6430 - loss: 1.0711 - val_accuracy: 0.6984 - val_loss: 0.8244
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 1049s 2s/step - accuracy: 0.7336 - loss: 0.7138 - val_accuracy: 0.7094 - val_loss: 0.8118
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 943s 2s/step - accuracy: 0.7855 - loss: 0.5759 - val_accuracy: 0.7050 - val_loss: 0.8361
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 1003s 2s/step - accuracy: 0.8402 - loss: 0.4399 - val_accuracy: 0.6982 - val_loss: 0.9680
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 966s 2s/step - accuracy: 0.8864 - loss: 0.3321 - val_accuracy: 0.6889 - val_loss: 1.0309


In [32]:
import numpy as np
y_pred_probs_l = LSTM_model.predict(padded_sequences_test)
y_pred_lstm = np.argmax(y_pred_probs_l, axis=1)   # predicted class (0–4)
y_true_lstm = np.argmax(test_labels, axis=1)

250/250 ━━━━━━━━━━━━━━━━━━━━ 61s 242ms/step


In [33]:
for i in range(60):
    print(f"Review {i+1}: True={y_true_lstm[i]}, Pred={y_pred_lstm[i]}")

Review 1: True=4, Pred=4
Review 2: True=4, Pred=4
Review 3: True=4, Pred=4
Review 4: True=4, Pred=4
Review 5: True=4, Pred=4
Review 6: True=4, Pred=4
Review 7: True=4, Pred=4
Review 8: True=4, Pred=4
Review 9: True=2, Pred=2
Review 10: True=4, Pred=4
Review 11: True=4, Pred=4
Review 12: True=4, Pred=4
Review 13: True=0, Pred=0
Review 14: True=1, Pred=1
Review 15: True=4, Pred=4
Review 16: True=3, Pred=3
Review 17: True=0, Pred=1
Review 18: True=4, Pred=4
Review 19: True=4, Pred=4
Review 20: True=1, Pred=1
Review 21: True=4, Pred=4
Review 22: True=4, Pred=4
Review 23: True=4, Pred=4
Review 24: True=3, Pred=4
Review 25: True=4, Pred=4
Review 26: True=4, Pred=4
Review 27: True=4, Pred=4
Review 28: True=4, Pred=4
Review 29: True=4, Pred=4
Review 30: True=4, Pred=4
Review 31: True=0, Pred=0
Review 32: True=4, Pred=2
Review 33: True=4, Pred=4
Review 34: True=4, Pred=4
Review 35: True=3, Pred=3
Review 36: True=4, Pred=4
Review 37: True=4, Pred=3
Review 38: True=4, Pred=4
Review 39: True=4, Pr

In [42]:
print_score(LSTM_model, padded_sequences_train, train_labels, padded_sequences_test, test_labels, train=True)

1000/1000 ━━━━━━━━━━━━━━━━━━━━ 437s 437ms/step
Train Result:
Accuracy Score: 93.40%
_______________________________________________
CLASSIFICATION REPORT:
                     0            1            2            3             4  \
precision     0.936713     0.889610     0.825506     0.905909      0.954573   
recall        0.935758     0.735680     0.849583     0.803744      0.988895   
f1-score      0.936235     0.805356     0.837372     0.851774      0.971431   
support    2942.000000  1676.000000  2400.000000  4540.000000  20442.000000   

           accuracy     macro avg  weighted avg  
precision  0.934031      0.902462      0.932945  
recall     0.934031      0.862732      0.934031  
f1-score   0.934031      0.880434      0.932466  
support    0.934031  32000.000000  32000.000000  
_______________________________________________
Confusion Matrix: 
[[ 2753    73    31     8    77]
 [  146  1233   221    23    53]
 [   15    61  2039   167   118]
 [    7    11   159  3649   714]


In [44]:
print_score(LSTM_model, padded_sequences_train, train_labels, padded_sequences_test, test_labels, train=False)

250/250 ━━━━━━━━━━━━━━━━━━━━ 61s 244ms/step
Test Result:
Accuracy Score: 68.89%
_______________________________________________
CLASSIFICATION REPORT:
                    0           1           2            3            4  \
precision    0.590343    0.321070    0.346870     0.382580     0.806330   
recall       0.514946    0.229117    0.341667     0.305727     0.877495   
f1-score     0.550073    0.267409    0.344249     0.339863     0.840409   
support    736.000000  419.000000  600.000000  1135.000000  5110.000000   

           accuracy    macro avg  weighted avg  
precision  0.688875     0.489438      0.666464  
recall     0.688875     0.453790      0.688875  
f1-score   0.688875     0.468400      0.675460  
support    0.688875  8000.000000   8000.000000  
_______________________________________________
Confusion Matrix: 
[[ 379   86   87   36  148]
 [  98   96   96   27  102]
 [  47   55  205  112  181]
 [  27   23   92  347  646]
 [  91   39  111  385 4484]]



In [45]:
new_sentence=['this product is bad and horrible i hate it','Good Quality and very Delight and Great product and recommended']
new_sentence=pd.Series(new_sentence)
new_sentence=new_sentence.apply(clean_text)
new_sentence=new_sentence.apply(remove_punctuation)
new_sentence=[stop(t) for t in new_sentence]
new_sequences=tokenizer.texts_to_sequences(new_sentence)
new_padded_sequences=pad_sequences(new_sequences,maxlen=padded_sequences_train.shape[1])
predictions_l=LSTM_model.predict(new_padded_sequences)
predicted_classes_lstm = np.argmax(predictions_l, axis=1)
print(predicted_classes_lstm)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
[0 4]


In [47]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences

sentiment_map = {
    0: "1 Star (Very Negative)",
    1: "2 Stars (Negative)",
    2: "3 Stars (Neutral)",
    3: "4 Stars (Positive)",
    4: "5 Stars (Very Positive)"
}

def predict_sentence(new_sentence, rnn_model, lstm_model, tokenizer, maxlen):
    # Convert to Series for preprocessing pipeline
    new_sentence = pd.Series(new_sentence)

    # Apply your preprocessing steps
    new_sentence = new_sentence.apply(clean_text)
    new_sentence = new_sentence.apply(remove_punctuation)
    new_sentence = [stop(t) for t in new_sentence]  # stopword removal

    # Tokenize + pad
    new_sequences = tokenizer.texts_to_sequences(new_sentence)
    new_padded_sequences = pad_sequences(new_sequences, maxlen=maxlen)

    # Predictions
    rnn_pred = np.argmax(rnn_model.predict(new_padded_sequences), axis=1)[0]
    lstm_pred = np.argmax(lstm_model.predict(new_padded_sequences), axis=1)[0]

    # Show results
    print("Input Review:", new_sentence[0])
    print("----------------------------------------------------")
    print(f"RNN Model Prediction : {sentiment_map[rnn_pred]}")
    print(f"LSTM Model Prediction: {sentiment_map[lstm_pred]}")


In [48]:
rev=input("please enter your review")
predict_sentence(rev, rnn_model, LSTM_model, tokenizer, padded_sequences_train.shape[1])

please enter your reviewWorst experience ever, completely disappointed.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
Input Review: worst experience ever completely disappointed
----------------------------------------------------
RNN Model Prediction : 1 Star (Very Negative)
LSTM Model Prediction: 1 Star (Very Negative)
